## Assignment 4
- Darpan Gaur        CO21BTECH11004
- Aditya Bacharwar      ES21BTECH11003
- Bapatu Manoj Kumar Reddy      ES21BTECH11010

In [108]:
# import libraries
import numpy as np
import pandas as pd

Input data format: Input csv file with m+1 rows and n+1 columns
- The first row excluding the last element is the cost vector c of length n
- The last column excluding the top element is the constraint vector b of length m
- Rows two to m+1 and column one to n is the matrix A of size m*n

### Utils

In [109]:
def get_input(input_file_path):
    '''
    Read input file and return c, b, A
    '''
    # read input file
    df = pd.read_csv(input_file_path, header=None)

    # c -> first row, except last column
    c = df.iloc[0, :-1].values

    # b - The last column excluding the top element (len) = m
    # b -> next m rows, last column
    b = df.iloc[1:, -1].values

    # A -> next m rows, except last column
    A = df.iloc[1:, :-1].values

    return c, b, A

In [110]:
def compute_constraints(z, A, b, tight=True):
    '''
    Compute tight / untight constraints

    Returns:
        A_tight, b_tight or A_untight, b_untight
    '''
    rows, constraints = [], []
    for i in range(A.shape[0]):
        row = A[i]
        constraint = b[i]
        
        # check if row.dot(z) == constraint
        if np.isclose(np.dot(row, z), constraint) == tight:
            rows.append(row)
            constraints.append(constraint)
    return np.array(rows), np.array(constraints)

In [111]:
def compute_directions(A):
    '''
    Compute directions from tight constraints
    '''
    try:
        return -np.linalg.inv(A)
    except np.linalg.LinAlgError:
        raise ValueError('Unable to find inverse of A')

In [112]:
def find_possible_directions(c, directions):
    '''
    Find the first positive direction
    '''
    
    cost = np.dot(c, directions)
    positive_cost_indices = np.where(cost > 0)[0]
    if len(positive_cost_indices) == 0:
        return None, None
    return positive_cost_indices[0], directions[:, positive_cost_indices[0]]

In [113]:
def check_unboundedness(A, direction):
    '''
    Check if the problem is unbounded in the direction
    '''
    
    feasible_direction = A @ direction > 0
    if not np.any(feasible_direction):
        raise Exception('Problem is unbounded')
    return False

In [114]:
def intial_point(A, b, max_iters:int = 1000):
    '''
    Finds a random initial vertex that satisfies the constraints
    - Selects n random rows from A
    - checks if constraints are satisfied
    - if not, repeat the process untl max_iters
    - else return the point
    '''
    for i in range(max_iters):
        
        indices = np.random.choice(A.shape[0], A.shape[1], replace=False)
        # get A, B candidate
        A_candidate = A[indices]
        b_candidate = b[indices]

        # check if row are linearly independent
        if np.linalg.matrix_rank(A_candidate) == A.shape[1]:

            # find the point
            point = np.linalg.inv(A_candidate) @ b_candidate
            if np.all(A @ point <= b):
                return point
            
    raise Exception('Initial point not found')

In [115]:
# Handing Degeneracy: Course Notes, Lecture 8: Assumption 2 - small perturbations of the hyperplanes can remove the degeneracies.
def add_noise(x, noise_level=1e-5):
    '''
    Add noise to the constraints vector
    ''' 
    return x + np.random.uniform(0, noise_level, x.shape) + noise_level
    

In [116]:
def simplex_algorithm(z, c, b, A, max_iters = 10000):
    '''
    Simplex algorithm for maximizing objective function
    '''

    z_temp = z          # initial point
    degeneracy = False  # flag for deg
    solved = False      # flag for solution

    try:
        for _ in range(max_iters):
            A_tight, b_tight = compute_constraints(z_temp, A, b)

            # check if degeneracy
            if A_tight.shape[0] == 0 or A_tight.shape[0] != A_tight.shape[1]:
                degeneracy = True
                raise Exception('Degeneracy detected')
           
            A_untight, b_untight = compute_constraints(z_temp, A, b, tight=False)

            # get directions
            directions = compute_directions(A_tight)

            # find possible directions
            direction_index, direction = find_possible_directions(c, directions)

            # check if direction is None
            if direction is None:
                break

            # check if the problem is unbounded
            if (check_unboundedness(A, direction)):
                raise Exception('Problem is unbounded')
            
            # find the step size and take min eta > 0
            eta = (b_untight - np.dot(A_untight, z_temp)) / np.dot(A_untight, direction)
            eta_min = np.min(eta[eta>0])

            # update
            z_temp = z_temp + eta_min * direction

            # Output: vertex visited, the objective function
            print(f'Vertex: {z_temp}, Objective: {np.dot(c, z_temp)}')

    except Exception as e:
        print(e)
        if(degeneracy):
            return False, None, None
        return True, None, None
    solved = True
    return solved ,z_temp, np.dot(c, z_temp)

In [117]:
# set input file path
input_file_path = 'test1.csv'

# get input
c, b, A = get_input(input_file_path)

# print input
print('c:', c)
print('b:', b)
print('A:', A)

c: [1. 2.]
b: [  0.    -8.   -63.09   4.29]
A: [[  2.    3. ]
 [ -3.    1. ]
 [-17.   17. ]
 [  0.    0.1]]


In [118]:
n_iterations = 1000
iter_count = 0
solved = False
optimal_z = None
optimal_cost = None

# try finding the optimal solution n_iterations times
for i in range(n_iterations):
    z = intial_point(A, b)
    solved ,optimal_z, optimal_cost = simplex_algorithm(z, c, b, A)
    if solved:
        iter_count = i
        break
    else:
        print('Iteration:', i, ', perturbing noise to the constraints')
        b = add_noise(b)

if solved and optimal_z is not None:
    print('Optimal solution found with ', iter_count, 'th initial point')
    print('optimal_z:', optimal_z) 
    print('optimal_cost:', optimal_cost)
else:
    print('Failed to find the optimal solution')

Optimal solution found with  0 th initial point
optimal_z: [ 2.22670588 -1.48447059]
optimal_cost: -0.7422352941176471
